# Survival Analysis Lab

Complete the following exercises to solidify your knowledge of survival analysis.

In [9]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from lifelines import KaplanMeierFitter

cf.go_offline()

In [17]:
data = pd.read_csv('../data/attrition.csv')
data

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,0,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,0,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,0,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,0,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,0,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,0,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,0,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [92]:
data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager', 'StatusGender'],
      dtype='object')

## 1. Generate and plot a survival function that shows how employee retention rates vary by gender and employee age.

*Tip: If your lines have gaps in them, you can fill them in by using the `fillna(method=ffill)` and the `fillna(method=bfill)` methods and then taking the average. We have provided you with a revised survival function below that you can use for the exercises in this lab*

In [15]:
def survival(data, group_field, time_field, event_field):
    kmf = KaplanMeierFitter()
    results = []

    for i in data[group_field].unique():
        group = data[data[group_field]==i]
        T = group[time_field]
        E = group[event_field]
        kmf.fit(T, E, label=str(i))
        results.append(kmf.survival_function_)

    survival = pd.concat(results, axis=1)
    front_fill = survival.fillna(method='ffill')
    back_fill = survival.fillna(method='bfill')
    smoothed = (front_fill + back_fill) / 2
    return smoothed

In [29]:
rates = survival(data, 'Gender', 'Age', 'Attrition')

rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Retention Rates by Tenure')

## 2. Compare the plot above with one that plots employee retention rates by gender over the number of years the employee has been working for the company.

In [37]:
rates = survival(data, 'Gender', 'Age', 'TotalWorkingYears')

rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Retention Rates by Gender and years worked at the company')

## 3. Let's look at retention rate by gender from a third perspective - the number of years since the employee's last promotion. Generate and plot a survival curve showing this.

In [40]:
rates = survival(data, 'Gender', 'YearsSinceLastPromotion', 'Attrition')

rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Retention Rates by gender and last promotion')

## 4. Let's switch to looking at retention rates from another demographic perspective: marital status. Generate and plot survival curves for the different marital statuses by number of years at the company.

In [39]:
rates = survival(data, 'MaritalStatus', 'Age','YearsAtCompany')

rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Retention Rates by Marital Status and years at the company')

## 5. Let's also look at the marital status curves by employee age. Generate and plot the survival curves showing retention rates by marital status and age.

In [43]:
rates = survival(data, 'MaritalStatus', 'Age', 'Attrition')
rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Retention Rates by Marital Status and Age')

## 6. Now that we have looked at the retention rates by gender and marital status individually, let's look at them together. 

Create a new field in the data set that concatenates marital status and gender, and then generate and plot a survival curve that shows the retention by this new field over the age of the employee.

In [52]:
#aquí inserto una nueva columna generada a partir de otras dos ya existentes...
data['StatusGender']=data['MaritalStatus']+data['Gender']
rates = survival(data, 'StatusGender', 'Age', 'Attrition')
rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Retention Rates by Marital Status and Gender')


## 6. Let's find out how job satisfaction affects retention rates. Generate and plot survival curves for each level of job satisfaction by number of years at the company.

In [53]:
rates = survival(data, 'JobSatisfaction', 'YearsAtCompany', 'Attrition')
rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Job satisfaction by number of years at the company')

## 7. Let's investigate whether the department the employee works in has an impact on how long they stay with the company. Generate and plot survival curves showing retention by department and years the employee has worked at the company.

In [54]:
rates = survival(data, 'Department', 'YearsAtCompany', 'Attrition')
rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Retention by department and years at the company')

## 8. From the previous example, it looks like the sales department has the highest attrition. Let's drill down on this and look at what the survival curves for specific job roles within that department look like.

Filter the data set for just the sales department and then generate and plot survival curves by job role and the number of years at the company.

In [82]:
salesdept=[]
salesdept=(e for e in data if data.Department == 'Sales')
salesdept
#rates = survival(data, 'Department'=='Sales', 'Jobs', 'Attrition')
#rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
#            title='Retention on the Sales dept. by job roles and departments')

<generator object <genexpr> at 0x7f97d08fb318>

## 9. Let examine how compensation affects attrition.

- Use the `pd.qcut` method to bin the HourlyRate field into 5 different pay grade categories (Very Low, Low, Moderate, High, and Very High).
- Generate and plot survival curves showing employee retention by pay grade and age.

In [102]:
#primera parte...
hourly=[]
hourly=data.HourlyRate
data['PayGradeCategories']=pd.qcut(hourly,5)

#segunda parte...
rates = survival(data, 'PayGradeCategories', 'Age', 'Attrition')
rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Employee retention by pay grade and age')

## 10. Finally, let's take a look at how the demands of the job impact employee attrition.

- Create a new field whose values are 'Overtime' or 'Regular Hours' depending on whether there is a Yes or a No in the OverTime field.
- Create a new field that concatenates that field with the BusinessTravel field.
- Generate and plot survival curves showing employee retention based on these conditions and employee age.

In [104]:
#Create a new field whose values are 'Overtime' or 'Regular Hours' depending on whether there is a Yes or a No in the OverTime field.
for e in data.OverTime:
    if e=='Overtime':
        e='Yes'
    else:
        e='No'
data['Overtime-BusinessT']=data.OverTime+data.BusinessTravel
rates = survival(data, 'Overtime-BusinessT', 'Age', 'Attrition')
rates.iplot(kind='line', yTitle='Retention Rate',xTitle='Age',
            title='Employee retention based on these conditions and employee age')